In [1]:
import sys
import subprocess
sys.path.append('f:/seismic/code/core')
sys.path.append('/seismic/code/core')
sys.path.append('/kaggle/input/my-seismic-library/')
import kaggle_support as kgs
import importlib
import seis_model
import time
import pandas as pd
import numpy as np
#kgs.cache_dir_write = kgs.cache_dir_write+'/../cache_evaluate/'
#kgs.cache_dir_read = kgs.cache_dir_write

F:/seismic/models/brendan/*.pth
Loading:  F:/seismic/models/brendan\unet2d_caformer_seed3_epochbest.pt
n_models: 1


In [2]:
# Set up model
importlib.reload(seis_model)
model = seis_model.default_model()

styleB not active yet


In [3]:
data = kgs.load_all_train_data(validation_only=True)
model.models[1].run_in_parallel = True
data_out = model.infer(data)
model.models[1].run_in_parallel = False

Processing in parallel : 100%|█████████████████████████████████████████████████████| 5000/5000 [03:03<00:00, 27.30it/s]


In [4]:
#kgs.disable_caching = True
model.models[1].run_in_parallel = False
families = list(set([d.family for d in data]))
families.sort()
res = dict()
res['family'], res['score_brendan'], res['score_mine'], res['score_mine_max'], res['time_taken'] = [],[],[],[],[]
for f in families:
    res['family'].append(f)
    this_data_out = []
    for d in data_out:
        if d.family==f:
            this_data_out.append(d)
    data_out_brendan = model.models[0].infer(this_data_out)
    res['score_brendan'].append(kgs.score_metric(data_out_brendan,show_diagnostics=False)[0])
    #data_out = model.infer(this_data[:1])
    t=time.time()
    #model.infer(this_data_out[:2])
    res['time_taken'].append((time.time()-t)/len(this_data_out[:2])/60)
    score,_,res_all = kgs.score_metric(this_data_out,show_diagnostics=False)
    res['score_mine'].append(score)
    res['score_mine_max'].append(np.max(res_all))
    #print(pd.DataFrame(res))
df = pd.DataFrame(res)
print('')
print('')
print('Brendan local score: ', np.mean(df['score_brendan']))
print('My local score: ', np.mean(df['score_mine']))
print('Estimate public LB: ', 28.8*np.mean(df['score_mine'])/np.mean(df['score_brendan']))
df

Importing cache brendan: 100%|█████████████████████████████████████████████████████| 500/500 [00:00<00:00, 4291.61it/s]




Brendan local score:  24.106589620386238
My local score:  20.257204153240593
Estimate public LB:  24.201161956146564


,family,score_brendan,score_mine,score_mine_max,time_taken
0,CurveFault_A,3.759723,3.759611,29.155440,0.0
1,CurveFault_B,69.516079,69.516079,179.627755,0.0
2,CurveVel_A,9.439880,9.439880,57.809796,0.0
3,CurveVel_B,38.293812,38.293812,192.743878,0.0
4,FlatFault_A,2.377070,2.366938,20.262268,0.0
5,FlatFault_B,24.400707,24.400707,105.244286,0.0
6,FlatVel_A,1.161858,0.048429,0.742857,0.0
7,FlatVel_B,7.335419,0.012971,0.328571,0.0
8,Style_A,35.717543,5.669809,22.279968,0.0
9,Style_B,49.063807,49.063807,76.157942,0.0


In [5]:
# No extra model: 28.8
# FlatVel:
# StyleA: 